In [1]:
import numpy as np
import pandas as pd
import math
import decimal
from decimal import Decimal

In [2]:
S = int(input("Stock Price: "))
X = int(input("Strike Price: "))
u = float(input("u: "))
d = float(input("d: "))
r = 0.01 * float(input("r: "))
n = int(input("Periods: "))

R = float(Decimal(math.exp(r)).quantize(Decimal('0.00')))
p = (R-d) / (u-d)

# stockprice
stockprice_lst = []
for i in range(n+1):
    stockprice_lst.append([])
    if i == 0:
        stockprice_lst[i].append(S)
    else:
        stockprice_lst[i].append(stockprice_lst[i-1][0] * u)
        for j in range(i):
            stockprice_lst[i].append(stockprice_lst[i-1][j] * d)
# probability
probability_lst = []
for i in range(n+1):
    probability_lst.append([])
    if i == 0:
        probability_lst[i].append(1)
    else:
        prob = float(Decimal(p ** i).quantize(Decimal('0.000')))
        probability_lst[i].append(prob)
        for j in range(i):
            if j < i-1:
                prob = probability_lst[i-1][j] * (1-p) + probability_lst[i-1][j+1] * p
                prob = float(Decimal(prob).quantize(Decimal('0.000')))
                probability_lst[i].append(prob)
            else:    
                prob = float(Decimal((1-p) ** i).quantize(Decimal('0.000')))
                probability_lst[i].append(prob)

# callprice & putprice
callprice_lst = []
putprice_lst = []
for i in range(n+1):
    callprice_lst.insert(0, [])
    putprice_lst.insert(0, [])
    for j in range(n-i+1):
        if i == 0:
            callprice_lst[0].append(max(0, stockprice_lst[n-i][j] - X))
            putprice_lst[0].append(max(0, X - stockprice_lst[n-i][j]))
        else:
            cp = float(Decimal((callprice_lst[1][j] * p + callprice_lst[1][j+1] * (1-p)) / R).quantize(Decimal('0.000')))
            pp = float(Decimal((putprice_lst[1][j] * p + putprice_lst[1][j+1] * (1-p)) / R).quantize(Decimal('0.000')))
            callprice_lst[0].append(cp)
            putprice_lst[0].append(pp)

# hedge ratio
hedgeratio_lst = []
for i in range(n):
    hedgeratio_lst.insert(0, [])
    for j in range(n-i):
        hedge = (callprice_lst[n-i][j] - callprice_lst[n-i][j+1]) / (stockprice_lst[n-i][j] - stockprice_lst[n-i][j+1])
        hedge = float(Decimal(hedge).quantize(Decimal('0.00000')))
        hedgeratio_lst[0].append(hedge)

# index
ind1 = []
for i in range(n+1):
    ind1.append(str(i) + '-period')
ind2 = []
for i in range(n):
    ind2.append(str(i) + '-period')

# result
stockprice_pd = pd.DataFrame(stockprice_lst, index=ind1).T.fillna(' ')
probability_pd = pd.DataFrame(probability_lst, index=ind1).T.fillna(' ')
callprice_pd = pd.DataFrame(callprice_lst, index=ind1).T.fillna(' ')
putprice_pd = pd.DataFrame(putprice_lst, index=ind1).T.fillna(' ')
hedgeratio_pd = pd.DataFrame(hedgeratio_lst, index=ind2).T.fillna(' ')

print("Binomial process for the stock price")
print(stockprice_pd)

print("Probabilities in parentheses")
print(probability_pd)

print("Binomial process for the call price")
print(callprice_pd)

print("Binomial process for the put price")
print(putprice_pd)

print("Hedge ratios in parentheses")
print(hedgeratio_pd)

print('Call value = ', callprice_lst[0][0])
print('Put value= ', putprice_lst[0][0])

Stock Price: 160
Strike Price: 150
u: 1.5
d: 0.5
r: 18.232
Periods: 5
Binomial process for the stock price
  0-period 1-period 2-period 3-period 4-period  5-period
0      160      240      360      540      810    1215.0
1                80      120      180      270     405.0
2                         40       60       90     135.0
3                                  20       30      45.0
4                                           10      15.0
5                                                    5.0
Probabilities in parentheses
  0-period 1-period 2-period 3-period 4-period  5-period
0        1      0.7     0.49    0.343     0.24     0.168
1               0.3     0.42    0.441    0.412     0.360
2                       0.09    0.189    0.265     0.309
3                               0.027    0.076     0.133
4                                        0.008     0.028
5                                                  0.002
Binomial process for the call price
  0-period 1-period 2-period 3